In [ ]:
print("Installing FFMPEG...")

# FFMPEG is crucial for audio extraction
!apt-get update -y
!apt-get install -y ffmpeg

print("Installing Python Libraries...")

# Core ML/AI Libraries
!pip install git+https://github.com/openai/whisper.git
!pip install transformers accelerate
!pip install torch torchvision torchaudio
!pip install --upgrade sentencepiece # Ensure sentencepiece is installed for tokenization

# Video Processing and Utilities
!pip install moviepy
!pip install ultralytics
!pip install opencv-python
!pip install easyocr
!pip install pytube --upgrade # CRITICAL FIX for YouTube download errors

# Frontend and Public Access
!pip install streamlit
!pip install pyngrok
!pip install watchdog

Installing FFMPEG...
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,817 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,838 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease

In [ ]:
%%writefile youtube_downloader.py
# youtube_downloader.py

from pytube import YouTube
import os

def download_youtube_video(url: str, output_path: str = "youtube_video_download.mp4") -> str:
    try:
        # Check and remove existing file
        if os.path.exists(output_path):
            os.remove(output_path)

        yt = YouTube(url)
        yt.bypass_age_gate()

        # Try to get progressive MP4 (best for direct download and smaller size)
        stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

        if stream is None:
            # Fallback: get highest resolution, often needs merging (but pytube handles it here)
            stream = yt.streams.get_highest_resolution()

        if stream is None:
            raise Exception("Could not find any suitable stream for download. The video may be region-locked or private.")

        print(f"Downloading: {yt.title}")
        stream.download(filename=output_path)
        print(f"Download complete: {output_path}")
        return output_path

    except Exception as e:
        print(f"An error occurred during YouTube download: {e}")
        raise

Writing youtube_downloader.py


In [ ]:
%%writefile transcriber.py
# transcriber.py

import subprocess
import whisper
import os

def extract_audio(video_path: str, audio_path: str = "temp_audio.wav") -> str:
    # This must be outside the `try/except` for FFmpeg to run
    if os.path.exists(audio_path):
        os.remove(audio_path)

    # Command to extract audio using FFmpeg
    command = [
        "ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a", audio_path, "-y"
    ]
    # Uses subprocess to run the command line tool FFmpeg
    # check=True will raise an error if FFmpeg fails
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, check=True)
    return audio_path

def transcribe_audio(audio_path: str, model_size: str = "base", device: str = "cpu") -> tuple[str, str]:
    """Transcribes and returns the transcript and detected language code."""

    print(f"Loading Whisper model: {model_size} on device: {device}...")
    model = whisper.load_model(model_size, device=device)

    print("Starting transcription...")
    # The default setting in Whisper is to detect language automatically
    result = model.transcribe(audio_path)

    detected_lang_code = result.get('language', 'en')

    print(f"\n*******************")
    print(f"DETECTED LANGUAGE: {detected_lang_code.upper()}")
    print(f"*******************\n")

    # Clean up audio file
    if os.path.exists(audio_path):
        os.remove(audio_path)

    return result['text'], detected_lang_code

Writing transcriber.py


In [ ]:
%%writefile summarizer.py
# summarizer.py (Using BART for stable, clean summarization)

from transformers import pipeline
import torch

# Use the established BART large CNN model
BART_MODEL_NAME = "facebook/bart-large-cnn"

def summarize_text(
    text: str,
    model_name:str = BART_MODEL_NAME,
    max_length: int = 200,
    min_length: int = 50,
    language_code: str = "en", # Not directly used by BART, but good for context
    device: str = "cpu"
) -> str:

    # Determine device for pipeline: 0 for CUDA, -1 for CPU
    device_int = 0 if device == "cuda" and torch.cuda.is_available() else -1

    print(f"Loading Summarization Model ({model_name}) on pipeline device: {device_int}...")

    summarizer = pipeline(
        "summarization",
        model=model_name,
        tokenizer=model_name,
        device=device_int
    )

    # BART takes the text directly for summarization
    summary = summarizer(
        text,
        max_length=max_length,
        min_length=min_length,
        do_sample=False # Best for consistent, non-random summaries
    )

    return summary[0]['summary_text']

Writing summarizer.py


In [ ]:
%%writefile vision_processor.py
# vision_processor.py

import cv2
from ultralytics import YOLO
import easyocr
import numpy as np

def extract_vision_data(video_path: str, frames_to_sample: int = 30, device: str = "cpu") -> str:
    vision_data = []

    # Load YOLOv8 for object detection
    object_model = YOLO('yolov8n.pt')

    # Configure EasyOCR for text detection
    use_gpu_ocr = device == 'cuda'
    # EasyOCR is a bit limited in language, defaulting to English
    reader = easyocr.Reader(['en'], gpu=use_gpu_ocr)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return "No visual data could be extracted."

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_sample = max(1, frames_to_sample) # Ensure at least 1 frame is sampled

    # Calculate indices for even sampling
    if frame_count <= frames_to_sample:
        sample_indices = list(range(frame_count))
    else:
        sample_indices = np.round(np.linspace(0, frame_count - 1, frames_to_sample)).astype(int)

    print(f"Total frames: {frame_count}, Sampling {len(sample_indices)} frames...")

    for i in sample_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()

        if not ret:
            continue

        # 1. Object Detection (YOLOv8)
        # Use try/except for YOLO in case of device issues
        try:
            results = object_model(frame, device=device, verbose=False)
            detected_objects = []
            for r in results:
                for c in r.boxes.cls:
                    class_name = object_model.names[int(c)]
                    detected_objects.append(class_name)

            unique_objects = sorted(list(set(detected_objects)))
            if unique_objects:
                object_description = f"Objects detected: {', '.join(unique_objects)}."
                vision_data.append(object_description)
        except Exception as e:
            print(f"YOLOv8 Error on frame {i}: {e}")
            pass # Continue if one frame fails

        # 2. Text Detection (EasyOCR)
        try:
            # EasyOCR expects a standard numpy array, which cv2.read() provides
            ocr_results = reader.readtext(frame, detail=0)

            if ocr_results:
                # Filter out short or generic promotional text
                filtered_text = [
                    t.strip() for t in ocr_results
                    if len(t.strip()) > 3 and not any(word in t.lower() for word in ['sponsored', 'shop', 'buy', 'link', 'video', 'ad', 'skip', 'subscribe'])
                ]
                if filtered_text:
                    text_description = f"Text visible on screen: \"{' '.join(filtered_text)}\"."
                    vision_data.append(text_description)
        except Exception as e:
            print(f"EasyOCR Error on frame {i}: {e}")
            pass # Continue if one frame fails


    cap.release()

    # Combine all frame-level data into a single string
    final_vision_data = " ".join(vision_data)

    if not final_vision_data:
        final_vision_data = "No significant objects or text were detected on screen."

    return final_vision_data

Writing vision_processor.py


In [ ]:
%%writefile utils.py
# utils.py

def chunk_text(text: str, chunk_size: int = 2000, overlap: int = 200) -> list:
    chunks = []
    start = 0
    text_length = len(text)

    # Ensure overlap is not larger than chunk size
    overlap = min(overlap, chunk_size - 100)
    overlap = max(0, overlap)

    while start < text_length:
        end = min(start + chunk_size, text_length)
        chunk = text[start:end]
        chunks.append(chunk)

        # Move the start position forward, respecting overlap
        start += chunk_size - overlap

        # Stop condition check for the loop
        if start >= text_length:
            break

    return chunks

def chunked_summarizer(text: str, summarize_func, max_chunk_size: int = 2000) -> str:
    print(f"Text too large for single summary. Chunking text with size {max_chunk_size}...")

    # Using a 10% overlap is a good heuristic for summarization chaining
    text_chunks = chunk_text(text, chunk_size=max_chunk_size, overlap=int(max_chunk_size * 0.1))

    # Generate partial summaries
    partial_summaries = [summarize_func(chunk) for chunk in text_chunks]

    combined_summary_input = " ".join(partial_summaries)
    print(f"Partial summaries combined. Total length for final summary: {len(combined_summary_input)} characters.")

    # Generate final summary from partial summaries
    final_summary = summarize_func(combined_summary_input)
    return final_summary

Writing utils.py


In [ ]:
%%writefile main.py
# main.py

import os
from transcriber import extract_audio, transcribe_audio
from summarizer import summarize_text, BART_MODEL_NAME
from utils import chunked_summarizer
from vision_processor import extract_vision_data

def video_to_summary(
    video_path: str,
    model_size: str = "base",
    summarizer_model_name: str = BART_MODEL_NAME, # Uses BART
    use_chunking: bool = False,
    device: str = "cpu"
) -> str:

    audio_path = "temp_audio.wav"

    # 1. Extract Audio & Transcribe
    print("\n--- 1. Transcribing Audio ---")
    extract_audio(video_path, audio_path)
    transcript, detected_lang_code = transcribe_audio(audio_path, model_size=model_size, device=device)

    # 2. Extract Visual Data
    print("\n--- 2. Processing Visual Data ---")
    vision_data = extract_vision_data(video_path, frames_to_sample=30, device=device)

    # 3. Combine Transcript and Vision Data
    combined_text = f"SPEECH TRANSCRIPT: {transcript}\n\nVISUAL DATA: {vision_data}"

    print(f"\n--- Combined Input Text (Start) ---\n{combined_text[:500]}...\n---------------------------\n")

    # Inner function for summarization (to be passed to chunked_summarizer if needed)
    def perform_summary(txt):
        return summarize_text(
            txt,
            model_name=summarizer_model_name,
            language_code=detected_lang_code,
            device=device
        )

    # 4. Summarize Combined Text
    print("\n--- 3. Generating Summary ---")
    if use_chunking:
        # Using a typical max input size for BART-like models
        final_summary = chunked_summarizer(
            text=combined_text,
            summarize_func=perform_summary,
            max_chunk_size=1024 # Recommended maximum for BART is ~1024 tokens
        )
    else:
        final_summary = perform_summary(combined_text)

    # Clean up video file after processing (audio is cleaned in transcriber.py)
    # The cleanup logic for the video is moved to app.py for better safety within the web app context.

    return final_summary

Writing main.py


In [ ]:
%%writefile app.py
# app.py (FINAL UPDATED VERSION)

import streamlit as st
import os
import torch
from main import video_to_summary
from youtube_downloader import download_youtube_video

def main():
    st.set_page_config(page_title="Multi-Modal Video Summarizer AI", layout="wide")
    st.title("🎬 Multi-Modal Video Summarizer AI")
    st.markdown("Combines **Speech Transcription (Whisper)** and **Visual Analysis (YOLO/EasyOCR)** to generate a comprehensive summary.")
    st.markdown("---")

    # --- Device Configuration ---
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cuda":
        st.sidebar.success(f"⚡ GPU Detected! Using **{torch.cuda.get_device_name(0)}** for fast processing.")
    else:
        st.sidebar.error("⚠️ No GPU Detected. Processing will run on CPU and may be slow.")

    st.sidebar.markdown("---")
    st.sidebar.subheader("Configuration")
    model_size = st.sidebar.selectbox(
        "Whisper Model Size (Smaller = Faster/Less Accurate)",
        ["base", "small", "medium"],
        index=0 # Default to 'base'
    )
    # BART summarizer handles chunking automatically if text is too long (via utils.py)
    use_chunking = st.sidebar.checkbox("Enable Text Chunking for Long Videos (Recommended)", value=True)


    # --- Video Input Options ---
    col1, col2 = st.columns(2)
    video_path_to_process = None

    # Initial state tracking
    uploaded_file = None
    youtube_url = None

    with col1:
        st.subheader("1. Upload a Video File 📁")
        uploaded_file = st.file_uploader("Drag and drop your video here (mp4, mov, avi, mkv). Max 200MB recommended for Colab.", type=["mp4", "mov", "avi", "mkv"])

        if uploaded_file is not None:
            temp_path = "uploaded_video.mp4"
            with open(temp_path, "wb") as f:
                f.write(uploaded_file.read())

            # --- CRITICAL FIX: Verify saved file existence and size ---
            if os.path.exists(temp_path) and os.path.getsize(temp_path) > 0:
                video_path_to_process = temp_path
                st.success(f"File uploaded successfully! Size: {os.path.getsize(temp_path) / (1024*1024):.2f} MB")
            else:
                st.error("Error: Uploaded file was saved but appears to be empty or corrupted.")
                video_path_to_process = None

    with col2:
        st.subheader("2. Enter a YouTube URL 🔗")
        # Ensure that if a file is uploaded, the URL field is ignored initially
        youtube_url = st.text_input("Paste the YouTube video URL:", disabled=(video_path_to_process is not None and uploaded_file is not None))

        if youtube_url and video_path_to_process is None: # Only download if no file was uploaded
            try:
                temp_path = "youtube_video_download.mp4"
                with st.spinner("Downloading YouTube video... (This might take a moment)"):
                    # The download function handles removal of old files
                    downloaded_path = download_youtube_video(youtube_url, temp_path)

                # --- CRITICAL FIX: Verify downloaded file existence and size ---
                if os.path.exists(downloaded_path) and os.path.getsize(downloaded_path) > 0:
                    video_path_to_process = downloaded_path
                    st.success("YouTube video downloaded! Ready for processing.")
                else:
                    st.error("Error: Download failed or resulted in an empty file. Check the URL validity.")
                    video_path_to_process = None

            except Exception as e:
                st.error(f"Error downloading YouTube video. Detailed error: {e}")
                video_path_to_process = None

    st.markdown("---")

    # --- Processing Logic ---
    if video_path_to_process is not None:

        st.subheader("Video to Summarize")
        st.video(video_path_to_process)
        st.markdown("---")

        if st.button("Start Multi-Modal Summarization", type="primary"):

            with st.spinner(f"Transcribing, detecting objects/text, and generating summary on **{device.upper()}** (Whisper Model: {model_size}). This may take a few minutes..."):
                try:
                    summary_result = video_to_summary(
                        video_path=video_path_to_process,
                        model_size=model_size,
                        use_chunking=use_chunking,
                        device=device
                    )

                    st.subheader("Final Multi-Modal Summary")
                    st.markdown("This summary is generated from both the **speech transcript** and **visual data (objects/text)** detected in the video.")
                    st.info(summary_result)

                except Exception as e:
                    st.error(f"An error occurred during processing. Please check the video file/URL and ensure the environment is set up correctly (e.g., FFMPEG, correct ngrok token). Detailed error: {e}")

                finally:
                    # Clean up all created temporary files
                    if os.path.exists(video_path_to_process):
                        os.remove(video_path_to_process)
                    if os.path.exists("temp_audio.wav"):
                        os.remove("temp_audio.wav")
                    st.markdown("---")
                    st.success("Processing complete and temporary files cleaned up.")


if __name__ == "__main__":
    main()

Writing app.py


In [ ]:
from pyngrok import ngrok
import subprocess
import os
import time

# 1. SET YOUR NGROK AUTH TOKEN HERE
# REPLACE THE TEXT BELOW WITH YOUR ACTUAL NGROK TOKEN!
NGROK_AUTH_TOKEN = "35QJYhA9ATuxG4jCzESK9eitksy_3JVo1VEnEfqAy6UX6eM8E"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Ensure Streamlit is not already running
subprocess.run(['pkill', '-f', 'streamlit'], check=False)
time.sleep(1)

# 2. Start ngrok tunnel for port 8501
# Check if a tunnel is already running and close it (safer cleanup)
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

public_url = ngrok.connect(8501, bind_tls=True).public_url
print(f"🚀 Streamlit App is running! Click the URL below:")
print(f"🔗 {public_url}")

# 3. Run the Streamlit app in the background
# The stdout/stderr is redirected to avoid clutter, but the process runs.
subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501', '--server.headless', 'true'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# This cell must remain running to keep the app alive!
print("\n* NOTE: This cell must remain running to keep the app alive. DO NOT STOP IT. *")

🚀 Streamlit App is running! Click the URL below:
🔗 https://diastral-overelegant-stefani.ngrok-free.dev

* NOTE: This cell must remain running to keep the app alive. DO NOT STOP IT. *
